# DEM correlation with NDVI 

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [1]:
import geemap, ee, eemont

In [2]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

## Get collections and filter

In [3]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

In [4]:
# Region of interest.
roi = ee.Geometry.Polygon([[[17.258492,60.457218],[17.212143,60.454001],[17.188797,60.439438],[17.19017,60.417751],[17.191544,60.398084],[17.208023,60.392996],[17.20047,60.383496],[17.220383,60.36975],[17.239609,60.377727],[17.254715,60.389434],[17.267418,60.411479],[17.272911,60.42436],[17.276688,60.440116],[17.258492,60.457218]]])

start_date = '2023-07-01'
stop_date = '2023-08-30'

In [5]:
s2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

s2_filtered = s2_collection.filterDate(start_date, stop_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filter(ee.Filter.bounds(roi)) \
    .map(maskS2clouds).spectralIndices(['NDVI','NDMI'])

# Get the number of images.
count = s2_filtered.size()
display('Count:', count)

resample = True
if resample:
    s2_rs = s2_filtered.map(lambda i: i.resample('bicubic'))
    
image = s2_rs.median()

'Count:'

## Density calculation

In [6]:
hn_min = 0.5
hn_max = 1
hn = 0.2
density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff']}

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
palette = {"min":0, "max":1, 'palette':color}

# initialize our map
ma = geemap.Map()
ma.centerObject(roi, 14)
ma.addLayer(image.select('NDVI'), palette, "NDVI")

ndvi_contours = geemap.create_contours(image.select('NDVI'), hn_min, hn_max, hn, region=None)
ma.addLayer(ndvi_contours, {'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}, 'contours', True)
ma.add_colorbar(density_vis, label="NDVI density", layer_name="NDVI", orientation="vertical", transparent_bg=True,)
ma

Map(center=[60.41846721355178, 17.230541532819135], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
ndvi_contours = geemap.create_contours(image.select('NDVI'), hn_min, hn_max, hn, region=None)
ndmi_contours = geemap.create_contours(image.select('NDMI'), hn_min, hn_max, hn, region=None)

display(ndvi_contours)
display(ndmi_contours)

## Clustering using NDVI density calculation 

In [8]:
training = ndvi_contours.sample(
    region = roi,
    scale = 10,
    numPixels = 10e9,
    geometries=True
    )

kmeans = ee.Clusterer.wekaKMeans(nClusters = 8, init = 2, distanceFunction = 'Euclidian', maxIterations = 500).train(training)
kmean_rs = image.cluster(kmeans)

# Visualize
nClusters=8
import geemap
m = geemap.Map()
# Display the RGB cluster means.
kmeans_vis = {'min': 0, 'max': nClusters - 1, 'palette': [ '#1E90FF', '#BDB76B', '#BC8F8F', '#FF4500', '#228B22']}

m.centerObject(roi,14)
m.addLayer(kmean_rs.visualize(**kmeans_vis).clip(roi), {}, 'Unsupervised K-means Classification', True, 1)
m

Map(center=[60.41846721355178, 17.230541532819135], controls=(WidgetControl(options=['position', 'transparent_…

## DEM 

In [68]:
cop_dem = ee.ImageCollection('COPERNICUS/DEM/GLO30')
cop_elevation = cop_dem.select('DEM')
# Reproject an image mosaic using a projection from one of the image tiles,
# rather than using the default projection returned by .mosaic().
proj = cop_elevation.first().select(0).projection()
slopeReprojected =  (cop_elevation.mosaic() \
                             .setDefaultProjection(proj)).resample('bicubic')

# Reduce the collection with a median reducer.
dem_elevation = slopeReprojected.reduce(ee.Reducer.mean())

h_min = 45
h_max = 55
h = 2
elevationVis = {
  'min': h_min,
  'max': h_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff'],
}
md = geemap.Map()
md.setCenter(17.223191, 60.436152, 14)
#md.addLayer(image, RGB_vis, "True color: Resample", True)
md.addLayer(dem_elevation, elevationVis, 'DEM')

contours = geemap.create_contours(dem_elevation, h_min, h_max, h, region=None)
md.addLayer(contours, {'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}, 'contours', True)
md.add_colorbar(elevationVis, label="Elevation (m)", layer_name="COP DEM", orientation="vertical", transparent_bg=True,)
md

Map(center=[60.436152, 17.223191], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [69]:
rgb_vis = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']}
mac = geemap.Map()
mac.centerObject(roi, 14)
mac.addLayer(image, rgb_vis, 'RGB')
left_layer = geemap.ee_tile_layer(image, rgb_vis, 'RGB' )
right_layer = geemap.ee_tile_layer(contours, {'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}, 'contours', True)

mac.split_map(left_layer, right_layer, left_label='RGB', right_label='NDVI')
mac.add_colorbar(elevationVis, label="Elevation (m)", layer_name="COP DEM", orientation="vertical", transparent_bg=True,)
mac

Map(center=[60.42909015971474, 17.23471649999347], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## Test correlation on DEM & NDVI / NDMI

In [70]:
display(ndvi_contours)
display(dem_elevation)

def normalize_min_max(image, band, roi):
    min_val = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    ).getInfo()[band]  # Remplacez 'band_name' par le nom de la bande de votre image

    max_val = image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    ).getInfo()[band]  # Remplacez 'band_name' par le nom de la bande de votre image

    normalized_image = image.subtract(min_val).divide(ee.Number(max_val).subtract(min_val))
    return normalized_image

dem_elevation = normalize_min_max(image, band, roi)

NameError: name 'band' is not defined

In [76]:
maxGap = 0
windowSize = 3
maxMaskedFrac = 0 # Fraction maximale de pixels masqués autorisés

# Combiner les deux images en une image à deux bandes
cross_cor = ee.Algorithms.CrossCorrelation(ee.Image(dem_elevation.select('mean').clip(roi)), ee.Image(ndmi_contours.select('NDMI').clip(roi)), maxGap, windowSize, maxMaskedFrac)	
display(cross_cor)
# Afficher les résultats

In [77]:
delta_x = cross_cor.select('deltaX')
delta_y = cross_cor.select('deltaY')
distance = cross_cor.select('euclideanDist')
correlation = cross_cor.select('correlationCoeff')

In [78]:
roi =  ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])

 
# Définir les paramètres de visualisation pour les différents calques
elevationVis = {
    'min': 45,
    'max': 55,
    'palette': ['f5f5dc', 'd2b48c', 'a0522d', '8b4513', '654321']
}

ndviContoursVis = {
    'palette': ['blue']
}

correlationVis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'cyan', 'white', 'yellow', 'red']
}

# Créer une carte geemap
map = geemap.Map()
map.centerObject(roi, 14)

# Ajouter les calques avec les nouvelles palettes de couleurs
map.addLayer(dem_elevation.clip(roi), elevationVis, 'DEM')
map.addLayer(ndmi_contours.clip(roi), ndviContoursVis, 'Contours', True)
map.addLayer(correlation, correlationVis, 'Correlation')

# Ajouter les contrôles de couche à la carte
map.addLayerControl()
map.add_colorbar(vis_params=elevationVis, label="Elevation (m)", position='bottomright')

# Afficher la carte
map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…